In [ ]:
import os
import csv
from typing import List
import psycopg2
from psycopg2.extras import execute_batch

PG_HOST = "34.87.68.164"
PG_PORT = 5432
PG_DBNAME = "postgres"
PG_USER = "adventureworks-operations"
PG_PASSWORD = "Nguyenhoangvu123@"

PG_SCHEMA = "production"
PG_TABLE = "product_document"

CSV_PATH = "./production_product_document.csv"

# === BẢNG MỤC TIÊU: DANH SÁCH CỘT THEO ĐÚNG THỨ TỰ ===
TABLE_COLUMNS: List[str] = [
    "product_id",
"document_node",
"modified_date"
]


def get_connection():
    return psycopg2.connect(
        host=PG_HOST,
        port=PG_PORT,
        dbname=PG_DBNAME,
        user=PG_USER,
        password=PG_PASSWORD,
        sslmode="require"   # Cloud SQL yêu cầu SSL
    )


def import_csv():
    if not os.path.exists(CSV_PATH):
        raise FileNotFoundError(f"Không tìm thấy file CSV: {CSV_PATH}")

    conn = get_connection()
    conn.autocommit = False
    cursor = conn.cursor()

    # Build câu INSERT
    cols_str = ", ".join(f'"{c}"' for c in TABLE_COLUMNS)
    placeholders = ", ".join(["%s"] * len(TABLE_COLUMNS))
    full_table_name = f'"{PG_SCHEMA}"."{PG_TABLE}"'
    insert_sql = f"INSERT INTO {full_table_name} ({cols_str}) VALUES ({placeholders})"

    print("Câu SQL:")
    print(insert_sql)

    batch_values = []
    batch_size = 500

    try:
        print("Đang đọc CSV (không header)...")

        with open(CSV_PATH, mode="r", encoding="utf-8") as f:
            reader = csv.reader(f)  # KHÔNG DÙNG DictReader

            for row in reader:
                if len(row) != len(TABLE_COLUMNS):
                    raise ValueError(
                        f"Dòng CSV không đúng số cột: {row}"
                    )

                batch_values.append(tuple(row))

                if len(batch_values) >= batch_size:
                    execute_batch(cursor, insert_sql, batch_values)
                    batch_values.clear()
                    print("Đã insert thêm 500 rows...")

            if batch_values:
                execute_batch(cursor, insert_sql, batch_values)

        conn.commit()
        print("🎉 Import CSV thành công!")

    except Exception as e:
        conn.rollback()
        print("❌ Lỗi, đã rollback transaction.")
        print("Chi tiết lỗi:", e)

    finally:
        cursor.close()
        conn.close()
        print("🔒 Kết nối đã đóng.")


if __name__ == "__main__":
    import_csv()


Câu SQL:
INSERT INTO "production"."product_document" ("product_id", "document_node", "modified_date") VALUES (%s, %s, %s)
Đang đọc CSV (không header)...
🎉 Import CSV thành công!
🔒 Kết nối đã đóng.
